In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import joblib
print(joblib.__version__)

1.3.2


In [ ]:
# 이미지 데이터를 저장할 리스트
images = []
labels = []

# 이미지가 들어있는 폴더 경로 지정
image_folder = "/content/drive/MyDrive/imdg/imdg1"

# 폴더 안의 이미지 파일 로드 및 클래스 목록 및 개수 추출
class_counts = {}
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg"):
        img = cv2.imread(os.path.join(image_folder, filename))
        if img is not None:
            # 이미지 크기 조정 또는 전처리 작업 수행
            # 예시: 이미지 크기 조정
            img = cv2.resize(img, (64, 64))
            images.append(img)
            # 클래스 레이블 할당 (가정: 파일 이름에서 클래스 정보 추출)
            # 파일 이름에서 클래스 정보 추출 (두 번째 밑줄 '_'까지 고려)
            class_name = filename.split("_")[0:1]  # 처음부터 두 번째 밑줄까지의 부분을 추출
            class_name = "_".join(class_name)  # 추출한 부분을 다시 밑줄로 연결하여 클래스 이름으로 사용
            labels.append(class_name)
            # 클래스 개수 카운트
            if class_name in class_counts:
                class_counts[class_name] += 1
            else:
                class_counts[class_name] = 1

# 클래스 목록과 클래스별 개수 출력
unique_labels = np.unique(labels)
print("클래스 목록 및 개수:")
for label in unique_labels:
    print(f"{label}: {class_counts[label]}")

# 레이블 정보를 이미지 파일 이름과 함께 저장할 파일 경로
label_file_path = "/content/drive/MyDrive/imdg/labels.txt"

# 이미지 파일 이름과 레이블 정보를 저장
with open(label_file_path, "w") as label_file:
    for filename, label in zip(os.listdir(image_folder), labels):
        if filename.endswith(".jpg"):
            label_file.write(f"{filename}: {label}\n")

print(f"레이블 파일이 {label_file_path}에 저장되었습니다.")




클래스 목록 및 개수:
EQ: 31
LQ: 31
Radioactive Material7.2: 31
coolant: 31
dangerous substances9-9: 31
dangerous substances9A: 31
flammable liquids3black: 31
flammable liquids3white: 31
flammable material4.1: 31
flammable material4.2: 31
flammable material4.3black: 31
flammable material4.3white: 31
fumigation: 31
gases2.1black: 31
gases2.1white: 31
gases2.2black: 31
gases2.2white: 31
gases2.3: 31
hazard division1.123: 31
hazard division1.4: 31
hazard division1.5: 31
hazard division1.6: 31
high-temp: 31
irreversible damage8: 31
marine pollutant: 31
oxidizing material5.1: 31
oxidizing material5.2black: 31
oxidizing material5.2white: 31
radioactive material7.1: 31
radioactive material7.3: 31
radioactive material7hack: 31
toxic material6.1: 31
toxic material6.2: 31
up1: 31
up2: 31
레이블 파일이 /content/drive/MyDrive/imdg/labels.txt에 저장되었습니다.


In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm  # tqdm 라이브러리 추가
import pickle  # joblib 라이브러리 추가

# 이미지 데이터를 저장할 리스트
images = []
labels = []

# 이미지가 들어있는 폴더 경로 지정
image_folder = "/content/drive/MyDrive/imdg/imdg1"

# 레이블 파일 경로
label_file_path = "/content/drive/MyDrive/imdg/labels.txt"

# 레이블 파일을 읽어서 이미지와 레이블을 매칭
with open(label_file_path, "r") as label_file:
    lines = label_file.readlines()
    for line in tqdm(lines, desc="Loading Images"):  # tqdm 사용하여 진행률 표시
        filename, label = line.strip().split(": ")
        img = cv2.imread(os.path.join(image_folder, filename))
        if img is not None:
            img = cv2.resize(img, (64, 64))
            images.append(img)
            labels.append(label)

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)
labels = np.array(labels)

# 데이터를 훈련 데이터와 테스트 데이터로 나눔
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 랜덤 포레스트 분류 모델 생성
rf_classifier = RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1)

# 모델 학습
rf_classifier.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 모델 평가
y_pred = rf_classifier.predict(X_test.reshape(X_test.shape[0], -1))
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

model_save_path = "/content/drive/MyDrive/imdg/rf_model.pkl"

# Model saving using pickle
with open(model_save_path, 'wb') as model_file:
    pickle.dump(rf_classifier, model_file)

print(f"모델이 {model_save_path}에 저장되었습니다.")

Loading Images: 100%|██████████| 1085/1085 [00:04<00:00, 251.29it/s]


Accuracy: 0.6451612903225806
                            precision    recall  f1-score   support

                        EQ       1.00      0.50      0.67         6
                        LQ       0.89      0.89      0.89         9
   Radioactive Material7.2       0.36      0.80      0.50         5
                   coolant       1.00      1.00      1.00         6
   dangerous substances9-9       0.88      0.78      0.82         9
    dangerous substances9A       1.00      0.71      0.83         7
   flammable liquids3black       0.29      0.50      0.36         4
   flammable liquids3white       1.00      0.40      0.57         5
     flammable material4.1       0.75      0.86      0.80         7
     flammable material4.2       1.00      1.00      1.00         3
flammable material4.3black       0.88      1.00      0.93         7
flammable material4.3white       1.00      1.00      1.00        10
                fumigation       1.00      0.38      0.55         8
             gases

In [8]:
 import os
import cv2
import numpy as np
import joblib

# 이미지가 들어있는 폴더 경로 지정
image_folder = "/content/drive/MyDrive/imdg/imdg1"

# 레이블 파일 경로
label_file_path = "/content/drive/MyDrive/imdg/labels.txt"

# 모델 경로
model_path = "/content/drive/MyDrive/imdg/rf_model.pkl"

# 레이블 파일을 읽어서 이미지와 레이블을 매칭
with open(label_file_path, "r") as label_file:
    lines = label_file.readlines()
    image_labels = {}
    for line in lines:
        filename, label = line.strip().split(": ")
        image_labels[filename] = label

# 모델 불러오기
loaded_rf_model = joblib.load(model_path)

# 예측을 확인할 이미지 파일명을 랜덤으로 선택 (예: 랜덤으로 3개 선택)
random_image_filenames = np.random.choice(list(image_labels.keys()), 3, replace=False)

# 선택한 이미지에 대한 예측 수행
for filename in random_image_filenames:
    img = cv2.imread(os.path.join(image_folder, filename))
    if img is not None:
        img = cv2.resize(img, (64, 64))
        img_flattened = img.reshape(1, -1)
        prediction = loaded_rf_model.predict(img_flattened)
        actual_label = image_labels[filename]
        print(f"Image: {filename}, Predicted Class: {prediction[0]}, Actual Class: {actual_label}")


Image: flammable material4.1_augmented9.jpg, Predicted Class: flammable material4.1, Actual Class: flammable material4.1
Image: radioactive material7.3_augmented3.jpg, Predicted Class: radioactive material7.3, Actual Class: radioactive material7.3
Image: gases2.3_augmented22.jpg, Predicted Class: gases2.3, Actual Class: gases2.3


In [ ]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt

# 이미지가 들어있는 폴더 경로 지정
image_folder = "/content/drive/MyDrive/imdg/imdg1"

# 레이블 파일 경로
label_file_path = "/content/drive/MyDrive/imdg/labels.txt"

# 모델 경로
model_path = "/content/drive/MyDrive/imdg/rf_model.pkl"

# 레이블 파일을 읽어서 이미지와 레이블을 매칭
with open(label_file_path, "r") as label_file:
    lines = label_file.readlines()
    image_labels = {}
    for line in lines:
        filename, label = line.strip().split(": ")
        image_labels[filename] = label

# 모델 불러오기
loaded_rf_model = joblib.load(model_path)

# 예측을 확인할 이미지 파일명을 랜덤으로 선택 (예: 랜덤으로 3개 선택)
random_image_filenames = np.random.choice(list(image_labels.keys()), 3, replace=False)


# 선택한 이미지에 대한 예측 수행
for filename in random_image_filenames:
    img = cv2.imread(os.path.join(image_folder, filename))
    if img is not None:
        img = cv2.resize(img, (64, 64))
        img_flattened = img.reshape(1, -1)
        prediction = loaded_rf_model.predict(img_flattened)
        actual_label = image_labels[filename]

        # Create a figure to display the images
        plt.figure(figsize=(10, 5))

        # Display the predicted image with the predicted label
        predicted_image = cv2.imread(os.path.join(image_folder, f"{prediction[0]}.jpg"))
        if predicted_image is not None:
            predicted_image = cv2.resize(predicted_image, (64, 64))
            plt.subplot(1, 2, 1)
            plt.imshow(cv2.cvtColor(predicted_image, cv2.COLOR_BGR2RGB))
            plt.title(f"Predicted: {prediction[0]}")

        # Display the original image with the filename
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"Actual: {actual_label} for {filename}")

        plt.show()


